In [1]:
import os, sys

if os.path.abspath(".") == "/content":  # google colab
    %pip install sktime keras==2.8.0 tensorflow==2.8.2 keras-nlp==0.4.0 mlflow GitPython tensorflow_addons==0.17.1 databricks-cli  --quiet
    from google.colab import drive

    drive.mount("/content/drive")

    os.chdir("/content/drive/My Drive/master-thesis/src")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.5/337.5 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 5.7 MB/s eta 0:00:

In [2]:
import sys, os
from models import Encoder_model
from preprocessing import ConstantLengthDataGenerator, plot
from mlflow_logging import MlFlowLogging
from reading import ConcatenatedDataset
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
import shutil
from sklearn.model_selection import train_test_split
import mlflow
import tensorflow as tf
import logging

In [3]:
!databricks configure --host https://community.cloud.databricks.com/

Username: pacyna1998@gmail.com
Password: 
Repeat for confirmation: 


In [4]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/pacyna1998@gmail.com/encoder")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1490200992843612', creation_time=1675518340194, experiment_id='1490200992843612', last_update_time=1675518340194, lifecycle_stage='active', name='/Users/pacyna1998@gmail.com/encoder', tags={'mlflow.experiment.sourceName': '/Users/pacyna1998@gmail.com/encoder',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'pacyna1998@gmail.com',
 'mlflow.ownerId': '8267380716084863'}>

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
mlflow_logging = MlFlowLogging()
mlflow.tensorflow.autolog()

In [6]:
category = "MOTION"

X, y = ConcatenatedDataset().read_dataset(dataset="ECG200")

INFO:root:Loading only one dataset: ECG200
INFO:root:Loading only one dataset: ECG200


In [7]:
y_encoder = sklearn.preprocessing.OneHotEncoder(categories="auto")
y = y_encoder.fit_transform(y.reshape(-1, 1)).toarray()
mlflow.log_param("y.shape", y.shape)
y.shape, X.shape

((200, 2), (200,))

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, stratify=y)

# Model

In [10]:
number_of_classes = y.shape[1]
output_directory = f"./data/models/encoder/category={category}"
batch_size = 256
input_length = 256
os.makedirs(output_directory, exist_ok=True)

In [11]:
input_layer = keras.layers.Input(shape=(input_length, 1))
fcn_model = Encoder_model(number_of_classes=y.shape[1])(input_layer)
model = keras.models.Model(inputs=input_layer, outputs=fcn_model)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4, decay_steps=3, decay_rate=1
)

callbacks = []
callbacks.append(
    tf.keras.callbacks.ModelCheckpoint(
        filepath=output_directory, monitor="val_accuracy", save_best_only=True
    )
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(lr_schedule),
    metrics=["accuracy"],
)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 256, 128)     768         ['input_1[0][0]']                
                                                                                                  
 instance_normalization (Instan  (None, 256, 128)    256         ['conv1d[0][0]']                 
 ceNormalization)                                                                                 
                                                                                                  
 p_re_lu (PReLU)                (None, 256, 128)     128         ['instance_normalization[0][0

In [13]:
kwargs = {
    "min_length": input_length,
    "max_length": input_length,
    "logging_call": mlflow.log_params,
}
data_generator_train = ConstantLengthDataGenerator(
    X_train,
    y_train,
    batch_size=batch_size,
    augmentation_probability=0.2,
    cutting_probability=0.2,
    padding_probability=0.5,
    **kwargs
)
data_generator_val = ConstantLengthDataGenerator(
    X_val,
    y_val,
    batch_size=len(y_val),
    augmentation_probability=0,
    cutting_probability=0,
    padding_probability=1,
    **kwargs
)
validation_data = next(data_generator_val)

In [14]:
history = model.fit(
    data_generator_train,
    epochs=2,
    validation_data=validation_data,
    callbacks=callbacks,
)

Epoch 1/2
 6/10 [=================>............] - ETA: 0s - loss: 0.5222 - accuracy: 0.7500

10/10 [==============================] - ETA: 0s - loss: 0.4835 - accuracy: 0.7508

INFO:tensorflow:Assets written to: ./data/models/encoder/category=MOTION/assets


10/10 [==============================] - 16s 623ms/step - loss: 0.4835 - accuracy: 0.7508 - val_loss: 0.1801 - val_accuracy: 0.9400
Epoch 2/2
10/10 [==============================] - ETA: 0s - loss: 0.2932 - accuracy: 0.8777

INFO:tensorflow:Assets written to: ./data/models/encoder/category=MOTION/assets


10/10 [==============================] - 4s 406ms/step - loss: 0.2932 - accuracy: 0.8777 - val_loss: 0.1543 - val_accuracy: 0.9800


INFO:tensorflow:Assets written to: /tmp/tmprof6rowu/model/data/model/assets
2023/02/04 15:43:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprof6rowu/model, flavor: tensorflow), fall back to return ['tensorflow==2.8.2']. Set logging level to DEBUG to see the full traceback.


# Logging the results

In [18]:
# mlflow_logging.log_confusion_matrix(*validation_data, classifier=model, y_encoder=y_encoder)
# mlflow_logging.log_history(
#     history.history, 
# )
mlflow_logging.log_example_data(*next(data_generator_train), encoder=y_encoder)

ValueError: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7efc58a02280> (for post_execute):


ValueError: ignored

ValueError: ignored

<Figure size 1080x720 with 1 Axes>

In [19]:
mlflow.end_run()